In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Scaling Criteo: Training with Merlin DLRM

This notebook is created using Tensorflow 20.05 version. In this notebook we converted the Tensorflow implementation of [Scaling Criteo](https://github.com/NVIDIA-Merlin/Merlin/blob/main/examples/scaling-criteo/03-Training-with-TF.ipynb) to Merlin Models. We have used DLRM model for this experiment.


## Overview

We observed that TensorFlow training pipelines can be slow as the dataloader is a bottleneck. The native dataloader in TensorFlow randomly sample each item from the dataset, which is very slow. The window dataloader in TensorFlow is also not much faster. In our experiments, we are able to speed-up existing TensorFlow pipelines by 9x using a highly optimized dataloader.


We have already discussed the NVTabular dataloader for TensorFlow in more detail in our [Getting Started with Movielens notebooks](https://github.com/NVIDIA-Merlin/NVTabular/tree/main/examples/getting-started-movielens).


We will use the same techniques to train a deep learning model for the [Criteo 1TB Click Logs dataset](https://ailab.criteo.com/download-criteo-1tb-click-logs-dataset/).


## Learning objectives
In this notebook, we learn how to:

- Use NVTabular dataloader with Merlin DLRM model using Criteo dataset
    
## NVTabular dataloader for TensorFlow
We’ve identified that the dataloader is one bottleneck in deep learning recommender systems when training pipelines with TensorFlow. The dataloader cannot prepare the next batch fast enough and therefore, the GPU is not fully utilized.

We developed a highly customized tabular dataloader for accelerating existing pipelines in TensorFlow. In our experiments, we see a speed-up by 9x of the same training workflow with NVTabular dataloader. NVTabular dataloader’s features are:

- removing bottleneck of item-by-item dataloading
- enabling larger than memory dataset by streaming from disk
- reading data directly into GPU memory and remove CPU-GPU communication
- preparing batch asynchronously in GPU to avoid CPU-GPU communication
- supporting commonly used .parquet format
- easy integration into existing TensorFlow pipelines by using similar API - works with tf.keras models


More information in our [blogpost](https://medium.com/nvidia-merlin/training-deep-learning-based-recommender-systems-9x-faster-with-tensorflow-cc5a2572ea49).

## Imports

In [ ]:
import os
import glob
import time
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

from merlin.schema import Tags
import tensorflow as tf

Define the path to directories which contains the processed data.

In [4]:
BASE_DIR = os.environ.get("BASE_DIR", "/workspace/criteo")
input_path = os.environ.get("INPUT_DATA_DIR", os.path.join(BASE_DIR, "test_dask/output"))

# path to processed data
PATH_TO_TRAIN_DATA = sorted(glob.glob(os.path.join(input_path, "train", "*.parquet")))
PATH_TO_VALID_DATA = sorted(glob.glob(os.path.join(input_path, "valid", "*.parquet")))

PATH_TO_TRAIN_DATA, PATH_TO_VALID_DATA

(['/workspace/criteo/test_dask/output/train/part_0.parquet'],
 ['/workspace/criteo/test_dask/output/valid/part_0.parquet'])

## Define hyperparameters

First, we define the data schema and differentiate between single-hot and multi-hot categorical features. Note, that we do not have any numerical input features.

In [5]:
CONTINUOUS_COLUMNS = ["I" + str(x) for x in range(1, 14)]
CATEGORICAL_COLUMNS = ["C" + str(x) for x in range(1, 27)]
LABEL_COLUMNS = ["label"]

BATCH_SIZE = int(os.environ.get("BATCH_SIZE", 64 * 1024))
EMBEDDING_SIZE = 32
EPOCHS = 1
LR = 0.01
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LR)

## Create the dataset

In this experiment, we will use [Dataset](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/io/dataset.py) which is a external-data wrapper for NVTabular.

In [6]:
train = Dataset(PATH_TO_TRAIN_DATA, part_mem_fraction=0.04)
valid = Dataset(PATH_TO_VALID_DATA, part_mem_fraction=0.04)

## Define the Model

Merlin models internally adds a dense layer with correct output based on the prediction tasks. In this case, since we are trying to implement Binary Classification task the dense layer will have a output dimension of 1. 

To know more about Merlin models go [here](https://github.com/NVIDIA-Merlin/models/tree/3b8e90368ba610011daacf87e78fbca73dae03c8/merlin/models/tf).

In [7]:
model = mm.DLRMModel(
    train.schema,                                                            # 1
    embedding_dim=EMBEDDING_SIZE,
    bottom_block=mm.MLPBlock([128, EMBEDDING_SIZE]),                         # 2
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(                            # 3
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    )               
)

Utilize the NVTabular data loader to create the validation dataloader which will be sent as a callback during training. The NVTabular data loader are initialized as usually and we specify both single-hot and multi-hot categorical features as cat_names. The data loader will automatically recognize the single/multi-hot columns and represent them accordingly.

In [8]:
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater

valid_dataloader = KerasSequenceLoader(
    valid,
    batch_size=BATCH_SIZE,
    label_names=LABEL_COLUMNS,
    cat_names=CATEGORICAL_COLUMNS,
    cont_names=CONTINUOUS_COLUMNS,
    engine="parquet",
    shuffle=False,
    parts_per_chunk=1,
)
validation_callback = KerasSequenceValidater(valid_dataloader)

## Compile and Train the model

In [9]:
%%time

model.compile(optimizer=OPTIMIZER, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=BATCH_SIZE, callbacks=[validation_callback], epochs=EPOCHS, verbose=1)

2022-06-08 16:09:22.676530: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-08 16:09:24.425708: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


11799/11800 [============================>.] - ETA: 0s - label/binary_classification_task/precision: 0.0316 - label/binary_classification_task/recall: 1.0189e-04 - label/binary_classification_task/binary_accuracy: 0.9679 - label/binary_classification_task/auc: 0.6346 - loss: 0.1412 - regularization_loss: 0.0000e+00 - total_loss: 0.1412

2022-06-08 16:17:30.566120: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/branch_executed/_19


{'val_label/binary_classification_task/precision': 0.097826086, 'val_label/binary_classification_task/recall': 1.7168699e-06, 'val_label/binary_classification_task/binary_accuracy': 0.9655388, 'val_label/binary_classification_task/auc': 0.66828704}
11800/11800 [==============================] - 671s 56ms/step - label/binary_classification_task/precision: 0.0316 - label/binary_classification_task/recall: 1.0189e-04 - label/binary_classification_task/binary_accuracy: 0.9679 - label/binary_classification_task/auc: 0.6346 - loss: 0.1412 - regularization_loss: 0.0000e+00 - total_loss: 0.1412 - val_label/binary_classification_task/precision: 0.0978 - val_label/binary_classification_task/recall: 1.7169e-06 - val_label/binary_classification_task/binary_accuracy: 0.9655 - val_label/binary_classification_task/auc: 0.6683 - val_loss: 0.2844 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.2844
CPU times: user 34min 59s, sys: 7min 11s, total: 42min 11s
Wall time: 11min 14s


## Evaluate the model

In our experiment, we used the following configurations:
```
BATCH_SIZE = int(os.environ.get("BATCH_SIZE", 64 * 1024))
EMBEDDING_SIZE = 64
EPOCHS = 10
LR = 0.08
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LR)

model = mm.DLRMModel(
    train.schema,                                                            # 1
    embedding_dim=EMBEDDING_SIZE,
    bottom_block=mm.MLPBlock([256, EMBEDDING_SIZE]),                         # 2
    top_block=mm.MLPBlock([256, 128, 64]),
    prediction_tasks=mm.BinaryClassificationTask(                            # 3
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    )               
)
```

and we achieved a AUC score of **0.7247** while using only 5 parquet files. 

In [10]:
eval_metrics = model.evaluate(valid, batch_size=BATCH_SIZE, return_dict=True)
eval_metrics

2322/2322 [==============================] - 58s 24ms/step - label/binary_classification_task/precision: 0.0978 - label/binary_classification_task/recall: 1.7169e-06 - label/binary_classification_task/binary_accuracy: 0.9655 - label/binary_classification_task/auc: 0.6683 - loss: 0.1440 - regularization_loss: 0.0000e+00 - total_loss: 0.1440


{'label/binary_classification_task/precision': 0.09782608598470688,
 'label/binary_classification_task/recall': 1.7168698605019017e-06,
 'label/binary_classification_task/binary_accuracy': 0.9655378460884094,
 'label/binary_classification_task/auc': 0.6682871580123901,
 'loss': 0.28435835242271423,
 'regularization_loss': 0.0,
 'total_loss': 0.28435835242271423}

## Save the model

In [ ]:
model.save(os.path.join(input_path, "model.savedmodel"))